In [1]:
from bs4 import BeautifulSoup as bs4
from bs4 import element as elm
import requests as req
from os import path

import tree_visualizer as tv
import tor_session as ts

In [4]:
matdump = open('./Matdump2.csv',encoding='UTF-8')
matdump.readlines(2)
firstmat = [s.strip() for s in matdump.readline().split(',',1)]
firstmat

['2dba74b7c80442dcb807276251777624',
 'Zoltek™ PX30 SW08 (Satin Weave) Carbon Fiber Fabric']

In [6]:
ses, tor_process=ts.make_session()
ses = req.Session()
ses.headers['User-agent'] = "Mozilla/5.0 (X11; Linux x86_64)"


started process.
C:\Program Files
Tor linked. Confirming connectivity.

LOCAL IP IS:
{
  "origin": "98.192.224.11"
}
 



Configuring socks...
REMOTE IP IS:
200
{
  "origin": "199.249.230.119"
}



In [7]:
resp = ses.get("http://www.matweb.com/search/datasheet_print.aspx?matguid="+firstmat[0])

In [8]:
print(resp.url)

http://www.matweb.com/search/datasheet_print.aspx?matguid=2dba74b7c80442dcb807276251777624


In [10]:
matsoup = bs4(resp.text, 'html.parser')
tables = matsoup.select('div[id*=DataSheet]>table')


In [11]:
class Properties:
        name = ""
        title_row = []
        entries = []
        
class Value:
    name = ""
    units = "ul"
    value = 0
    
    def __init__(self,name,value=None, units=None):
        if isinstance(name,str):
            if isinstance(value,(float,int,tuple,str)) and isinstance(units,(str)):
                if isinstance(value,tuple) and len(value) != 2:
                    raise ValueError("If a range is provided, a maximum and minimum must be given")
                self.name = name
                self.units = units
                self.value = value
            else:
                raise TypeError("Value must be a number or tuple, units must be a string")
        elif isinstance(name,elm.Tag):
            print("Tag Detected. Here's where I'd put the parser. IF I HAD ONE!")
        else:
            raise TypeError("Pass either a tag to parse or a value.")


class Material:
    def _parse_info(self,info_tag):
        # Take a provided tag object, and exctract data from it.
        rows = info_tag.select('tr')
        self.info_dict = dict()
        for row in rows:
            if row.th is not None:
                self.name = row.th.get_text()
                continue
            cells = row.select('td')
            if len(cells) != 2:
                continue
            prop_name = cells[0].get_text()
            prop_value = cells[1].get_text(",",strip=True)
            self.info_dict[prop_name] = prop_value
    
    
    def _parse_props(self,prop_tag):
        self.prop_dict = dict()
        category = None
        for row in prop_tag.select('tr'):
            if row.th is not None:
                category = row.th.string.replace("Properties","")
                self.prop_dict[row.th.string.replace("Properties","")]= set()
                continue
                
            if category is None or row.find(class_='dataCell') is None:
                continue
            
            print(row.attrs)

            self.prop_dict[category].add(Value(row))        

In [12]:
stuff = Material()
stuff._parse_info(tables[0])
#print(tables[1].find_all('tr'))
for item in tables[0].find_all('tr'):
    print(item.prettify(),"\n")

<tr>
 <th colspan="2" style="font-size:larger;">
  Zoltek™ PX30 SW08 (Satin Weave) Carbon Fiber Fabric
 </th>
</tr>
 

<tr id="ctl00_ContentPrint_ucDataSheet1_trMatlGroups">
 <td style="vertical-align:top;">
  <b>
   Categories:
  </b>
 </td>
 <td>
  <a href="/Search/MaterialGroupSearch.aspx?GroupID=283" title="See all materials in this category.">
   Carbon
  </a>
  ;
  <a href="/Search/MaterialGroupSearch.aspx?GroupID=319" title="See all materials in this category.">
   Carbon Fiber/Carbon Fibre
  </a>
  ;
  <a href="/Search/MaterialGroupSearch.aspx?GroupID=285" title="See all materials in this category.">
   Other Engineering Material
  </a>
  ;
  <a href="/Search/MaterialGroupSearch.aspx?GroupID=286" title="See all materials in this category.">
   Composite Fibers
  </a>
  <p>
  </p>
 </td>
</tr>
 

<tr id="ctl00_ContentPrint_ucDataSheet1_trMatlNotes">
 <td style="vertical-align:top;">
  <b>
   Material Notes:
  </b>
 </td>
 <td>
  ZOLTEK PX30 Woven Carbon Fabrics tailorability res

In [60]:
for key,value in  stuff.prop_dict.items():
    print(key)
    print(value)


Physical 
{<__main__.Value object at 0x7f0a125ded90>}
Electrical 
{<__main__.Value object at 0x7f0a125ded00>}
Component Elements 
{<__main__.Value object at 0x7f0a125de430>}
Descriptive 
{<__main__.Value object at 0x7f0a125dec10>, <__main__.Value object at 0x7f0a125de4c0>, <__main__.Value object at 0x7f0a125de4f0>}


In [18]:
for key,value in stuff.info_dict.items():
    print(key,"=\n",value,"\n-------------------------\n")

Categories:  =
 Carbon,;,Carbon Fiber/Carbon Fibre,;,Other Engineering Material,;,Composite Fibers 
-------------------------

Material Notes:  =
 ZOLTEK PX30 Woven Carbon Fabrics tailorability results from controlling the yield on rovings and yarns, and allows for a variety of finished composite thicknesses. The spun yarns have many surface fibrils that protrude in various directions at various lengths from the surface. This fiber loft contributes to
high cross-ply tensile and interlaminar shear strengths for 2-D composite materials. Contour conformation without wrinkling is another advantage of spun yarn fabrics. ZOLTEK PX30 materials undergo high temperature carbonization process, involving a long temperature soak time, resulting in a very thermally stable and chemically pure fabric with low oxidation rate.,Information provided by Zoltek Corporation. 
-------------------------

Vendors:  =
 No vendors are listed for this material. Please,click here,if you are a supplier and would li